# Init Cell

In [3]:
from unsloth import FastLanguageModel # API для подгрузки модели

# Параметры инициализации
max_seq_length = 4096
max_summary_length = 256
dtype = None
load_in_4bit = True

# Создаём модель и токинайзер
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen2.5-0.5B",
    attn_implementation = "flash_attention_2",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 02-15 11:08:21 __init__.py:190] Automatically detected platform cuda.
==((====))==  Unsloth 2025.2.9: Fast Qwen2 patching. Transformers: 4.48.3.
   \\   /|    GPU: NVIDIA GeForce RTX 4070 Ti. Max memory: 11.994 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = True]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [ ]:
# Оптимизируем модель через Lora
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None
)


Unsloth 2025.2.9 patched 24 layers with 24 QKV layers, 24 O layers and 24 MLP layers.


# Dataset processing

In [4]:
# Базовый alpaca prompting для SFT
alpaca_prompt = """Ниже приведена инструкция, описывающая задание, в сочетании с вводными данными, которые обеспечивают дальнейший контекст. Напишите ответ, который соответствующим образом выполнит запрос.

### Инструкция:
{}

### Вводные данные:
{}

### Ответ:
{}"""

# Функция перевода данных под промпт для модели на задачу "Резюмирования текста"
EOS_TOKEN = tokenizer.eos_token # конечный токен у токенайзера Qwen
def formatting_prompts_func(examples):
    inputs = examples["input"]
    outputs = examples["output"]
    texts = []
    # Создаём сэмпл по шаблонному промпту, в конце добавляем токен остановки
    for input, output in zip(inputs, outputs):
        text = alpaca_prompt.format("Резюмируй текст, сохрани важную информацию о нём в кратком объёме.", input, output) + EOS_TOKEN
        texts.append(text)
    return {"text": texts}

In [5]:
from datasets import load_dataset, concatenate_datasets # работа с датасетами HF

# Функция подготовки датасетов
def prepare_sets(ds_repo: str, name: str, revision: str, split: str):
    ds = load_dataset(ds_repo, name=name, revision=revision, split=split) # загрузка с сервера
    ds = ds.remove_columns([x for x in ds.column_names if x not in ('text', 'summary')]) # удаление лишних столбцов
    ds = ds.rename_columns({"text": "input", "summary": "output"})  # переименовываем столбцы под общий формат

    # Фильтруем данные по:
    ds = ds.filter(
        lambda x: 
            len(x["input"]) >= 512 and # 1) длина исходного текста >= 512 символов
            len(x["input"]) <= max_seq_length and  # 2) длина исходного текста <= 4096
            len(x["output"]) >= 20 and # 3) Резюмированный текст >= 20 символов
            len(x['output']) <= max_summary_length and  # 4) Резюмированный текст <= 128 символов
            len(x["output"])/len(x["input"]) < 0.3  # 5) Соотношение объёмов 2-х текстов <= 0.3
    )

    if split == 'test': # при тестовой выборке просто возвращаем датасет
        return ds
    
    ds = ds.map(formatting_prompts_func, batched=True) # в ином случае формируем промпт

    # Фильтруем данные, чтобы длина промпта <= 4096 символов
    ds = ds.filter(
        lambda x: len(x["text"]) <= max_seq_length,
    )
    return ds

In [6]:
train_dataset = concatenate_datasets(
    [
        prepare_sets(ds_repo='IlyaGusev/gazeta', name=None, revision="v2.0", split='train'), 
        prepare_sets(ds_repo='IlyaGusev/gazeta', name=None, revision="v2.0", split='validation'),
        prepare_sets(ds_repo='csebuetnlp/xlsum', name='russian', revision=None, split='train'),
        prepare_sets(ds_repo='csebuetnlp/xlsum', name='russian', revision=None, split='validation')
    ]
)
test_dataset = concatenate_datasets(
    [
        prepare_sets(ds_repo='IlyaGusev/gazeta', name=None, revision="v2.0", split='test'),
        prepare_sets(ds_repo='csebuetnlp/xlsum', name='russian', revision=None, split='test')
    ]
)

Filter:   0%|          | 0/60964 [00:00<?, ? examples/s]

Map:   0%|          | 0/9662 [00:00<?, ? examples/s]

Filter:   0%|          | 0/9662 [00:00<?, ? examples/s]

Filter:   0%|          | 0/6369 [00:00<?, ? examples/s]

Map:   0%|          | 0/678 [00:00<?, ? examples/s]

...

Filter:   0%|          | 0/7780 [00:00<?, ? examples/s]

Map:   0%|          | 0/4920 [00:00<?, ? examples/s]

Filter:   0%|          | 0/4920 [00:00<?, ? examples/s]

Filter:   0%|          | 0/6793 [00:00<?, ? examples/s]

Filter:   0%|          | 0/7780 [00:00<?, ? examples/s]

In [7]:
# Взгляем на объём датасетов
train_dataset, test_dataset

(Dataset({
     features: ['input', 'output', 'text'],
     num_rows: 38413
 }),
 Dataset({
     features: ['input', 'output'],
     num_rows: 5520
 }))

In [8]:
test_dataset['input'][0]

'В Екатеринбурге на 82-м году жизни скончался советский и российский детский писатель Владислав Крапивин. Об этом сообщили представители Минздрава региона. «Да. Сегодня», — цитирует пресс-службу Минздрава РИА «Новости». Согласно невестке Крапивина Ларисе, писатель умер утром 1 сентября по местному времени. «Умер в 06:40. Ему стало хуже, ночью отвезли в реанимацию — и все», — передает ее слова ТАСС. Ранее Лариса в своем Facebook рассказывала, что Крапивин был госпитализирован 15 июля. Как тогда подчеркивала родственница, решение об отправке писателя в госпиталь было принято на фоне того, что он «внезапно упал утром, встав с кровати». «Он был в сознании, хотя и плохо разговаривал. И было ощущение, что это инсульт. Была срочно вызвана скорая помощь. [В больнице ему] сделали КТ головного мозга и легких, кардиограмму и анализ крови, семье было сказано, что инсульта и инфаркта нет, но есть подозрение на COVID, потому что КТ легких показала наличие пневмонии с 25% поражения», — подчеркнула не

# SFT

In [ ]:
from trl import SFTTrainer, SFTConfig
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    max_seq_length = max_seq_length,
    args = SFTConfig(
        dataset_text_field = "text",
        overwrite_output_dir = True,
        num_train_epochs = 3,
        per_device_train_batch_size = 32,
        gradient_accumulation_steps = 4,
        learning_rate = 3e-4,
        logging_steps = 10,
        
        warmup_ratio = 0.1,
        lr_scheduler_type = "linear",
        optim = "adamw_torch_fused",
        weight_decay = 0.01,
        max_grad_norm = 0.8,
        
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",
        save_steps=200,
        torch_compile=True,
        gradient_checkpointing=True
    )
)

In [9]:
import torch
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA GeForce RTX 4070 Ti. Max memory = 11.994 GB.
0.547 GB of memory reserved.


In [10]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 38,413 | Num Epochs = 3
O^O/ \_/ \    Batch size per device = 32 | Gradient Accumulation steps = 4
\        /    Total batch size = 128 | Total steps = 900
 "-____-"     Number of trainable parameters = 8,798,208


Unsloth: Enabled auto compiling


Step,Training Loss
10,2.283700
20,2.182700
30,2.034600
40,1.943700
50,1.907900
60,1.893700
70,1.868600
80,1.863900
90,1.855300
100,1.835100


In [ ]:
# Сохраняем модель и токенайзер
model.save_pretrained("qwen2_5_lora_sft_model")
tokenizer.save_pretrained("qwen2_5_lora_sft_model")

('qwen2_5_lora_sft_model/tokenizer_config.json',
 'qwen2_5_lora_sft_model/special_tokens_map.json',
 'qwen2_5_lora_sft_model/vocab.json',
 'qwen2_5_lora_sft_model/merges.txt',
 'qwen2_5_lora_sft_model/added_tokens.json',
 'qwen2_5_lora_sft_model/tokenizer.json')

In [ ]:
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory*100, 3)


print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

12916.6526 seconds used for training.
215.28 minutes used for training.
Peak reserved memory = 4.072 GB.
Peak reserved memory for training = 3.525 GB.
Peak reserved memory % of max memory = 33.95 %.
Peak reserved memory for training % of max memory = 29.39 %.


# Inference cell

In [ ]:
def inference(input_txt: str, _model, _tokenizer):
    _model = FastLanguageModel.for_inference(_model)
    # Токенизируем батч длинной 1 из формированного промпта при обучении, "Ответ: " оставляем пустым для генерашки
    inputs = _tokenizer(
    [
        alpaca_prompt.format(
            "Резюмируй текст, сохрани важную информацию о нём в кратком объёме.",
            input_txt,
            "",
        )
    ], return_tensors = "pt").to("cuda")

    # Генерим выхлоп модели и возвращаем декодированное сообщение. Не забываем делать split, чтобы не выводить весь промпт
    outputs = _model.generate(input_ids = inputs.input_ids, attention_mask = inputs.attention_mask,
                            max_new_tokens=max_summary_length, use_cache=True)
    return _tokenizer.batch_decode(outputs, skip_special_tokens=True)[0].split("### Ответ:\n")[-1]

In [ ]:
# Грузим базовую предобученную модель для теста
model_base, tokenizer_base = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen2.5-0.5B",
    attn_implementation = "flash_attention_2",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit
)

# Грузим SFT модель для теста
model_sft, tokenizer_sft = FastLanguageModel.from_pretrained(
    model_name = "qwen2_5_lora_sft_model",
    attn_implementation = "flash_attention_2",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit
)

==((====))==  Unsloth 2025.2.9: Fast Qwen2 patching. Transformers: 4.48.3.
   \\   /|    GPU: NVIDIA GeForce RTX 4070 Ti. Max memory: 11.994 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = True]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
==((====))==  Unsloth 2025.2.9: Fast Qwen2 patching. Transformers: 4.48.3.
   \\   /|    GPU: NVIDIA GeForce RTX 4070 Ti. Max memory: 11.994 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = True]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


# Test time

In [ ]:
from pprint import pprint # красивый вывод
from random import choice # получение рандомного сэмпла из выборки

# Формируем случайный сэмпл из тестовой выборки
random_idx = choice(range(0, len(test_dataset)))
random_text = test_dataset['input'][random_idx]
random_output = test_dataset['output'][random_idx]

# Генерируем ответ у 2-х моделей
predicted_output_pretrained = inference(random_text, model_sft, tokenizer_sft)
predicted_output_base = inference(random_text, model_base, tokenizer_base)

# Вывод выхлопа моделей и ground truth
print('-' * 50)
print("SFT model: ")
print('-' * 50)
pprint(predicted_output_pretrained)

print('-' * 50)
print("Base model: ")
print('-' * 50)
pprint(predicted_output_base)

print('-' * 50)
print('Ground truth: ')
print('-' * 50)
pprint(random_output)

--------------------------------------------------
SFT model: 
--------------------------------------------------
('Полиция Украины задержала политика Михаила Саакашвили, которого обвиняют в '
 'содействии преступным организациям и посягательстве на территориальную '
 'целостность Украины.')
--------------------------------------------------
Base model: 
--------------------------------------------------
('После задержания политика Саакашвили в киевском ресторане "Сулугуни" был '
 'задержан и отправлен в бус, затем повезли и затем посадили в вертолет. '
 'Вертолет, как я понимаю, некоторое время кружил над Киевом. После чего '
 'приземлился в Борисполе, где меня, скрутив руки и с применением грубой силы, '
 'посадили в самолет.')
--------------------------------------------------
Ground truth: 
--------------------------------------------------
('Бывший президент Грузии и экс-губернатор Одесской области Украины Михаил '
 'Саакашвили, который в последнее время регулярно выступает с крит

# Results

По окончанию SFT я в целом доволен результатом:
1) Модель научилась следовать формату промптов
2) Лучше понимает русский текст
3) Улавливает контекст при резюмировании
4) Хоть ответы и отличаются от GT, но при этом главная мысль текста передана

## Сравнивая базовый **Qwen2.5-0.5B** и **SFT Qwen2.5-0.5B**:
* Базовая модель довольно плохо следует инструкциям и не делает резюмирование текста, что часто обрезается токенайзером по длине строки. Не соблюдает формат промпта, что делает невозможным правильно парсить ответ. Бывает часто зацикливается на повторении, если не ставить большой ```reputatuin_penality```.

* В свою очередь SFT модель понимает свою задачу, делает правильные резюмирования текстов и соблюдает формат вывода. Всегда укладывается в нужное количество новых токенов при генерации. Не зацикливается при генерации ответа.

## Говоря о плюсах SFT подхода для трансформерных моделей:
1) Малое потребление VRAM (спасибо за поддержку квантизации)
2) Довольно быстрое обучение (спасибо, за оптимизацию от unsloth)
3) Нетребовательна к gold-данным, так как тонкая настройка не сильно портит модель, но при этом хоть какая-то полезная дата бустит модель.

## Говоря о минусах SFT подхода для трансформерных моделей:
1) Нужно много данных (как собственно и для всего обучения трансформерных моделек)
2) Полностью изменить спецификацию моделей, без дополнительной RL настройки после SFT (например PPO/GRPO и прочие policy) вряд ли выйдет.
3) Лучше показывают себя LLM большего размера, хотя бы с >1.5B параметрами, так как они быстрее улавливают задачу и контекст.